# Real-world Applications of ML in Signal Processing: Image and Video Processing
## Using Generative Adversarial Networks (GANs) in TensorFlow

In this notebook, we'll explore how to build and train a Generative Adversarial Network (GAN) for image generation using TensorFlow. GANs are powerful tools for generating synthetic data that resembles the training data distribution, which can be used in various applications such as image synthesis, style transfer, and more.

## Introduction to Generative Adversarial Networks (GANs)

Generative Adversarial Networks (GANs) are deep learning models designed to generate new data samples that resemble the training data. GANs consist of two neural networks, a generator and a discriminator, which compete against each other in a game-theoretic framework.

## Installing Necessary Libraries

Let's start by installing the necessary libraries. We will use TensorFlow, Matplotlib, and other essential libraries.

In [1]:
!pip install tensorflow matplotlib numpy pillow

## Importing Libraries

Next, we'll import the required libraries.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

## Building the Generative Adversarial Network (GAN)

We'll now define the generator and discriminator networks.

In [3]:
latent_dim = 100

generator = keras.Sequential([
    keras.Input(shape=(latent_dim,)),
    layers.Dense(128 * 16 * 16),
    layers.Reshape((16, 16, 128)),
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu'),
    layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', activation='relu'),
    layers.Conv2D(3, kernel_size=5, padding='same', activation='sigmoid')
], name='generator')

discriminator = keras.Sequential([
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),
    layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'),
    layers.GlobalMaxPooling2D(),
    layers.Dense(1, activation='sigmoid')
], name='discriminator')

gan = keras.Sequential([generator, discriminator])

discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
                      loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False
gan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
            loss='binary_crossentropy')

gan.summary()

## Loading and Preprocessing Data

For this example, we'll use the CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes. We'll preprocess the data and prepare it for training and evaluation.

In [4]:
(x_train, _), (x_test, _) = keras.datasets.cifar10.load_data()

# Normalize data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Visualize some examples
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(x_train[i])
    plt.axis('off')
plt.suptitle('Examples from CIFAR-10 dataset')
plt.show()

## Training the GAN

We'll train the GAN on the CIFAR-10 dataset.

In [5]:
epochs = 50
batch_size = 64

for epoch in range(epochs):
    for batch in range(x_train.shape[0] // batch_size):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
        combined_images = np.concatenate([real_images, generated_images])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        labels += 0.05 * np.random.random(labels.shape)
        d_loss = discriminator.train_on_batch(combined_images, labels)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.zeros((batch_size, 1))
        a_loss = gan.train_on_batch(noise, misleading_targets)

    print(f'Epoch {epoch+1}, Discriminator Loss: {d_loss[0]}, Discriminator Accuracy: {100*d_loss[1]}%')
    print(f'Epoch {epoch+1}, Adversarial Loss: {a_loss}')

    # Visualize generated images
    if epoch % 10 == 0:
        plt.figure(figsize=(10, 10))
        for i in range(25):
            plt.subplot(5, 5, i+1)
            plt.imshow(generated_images[i])
            plt.axis('off')
        plt.suptitle(f'Generated Images - Epoch {epoch+1}')
        plt.show()

## Conclusion

In this notebook, we explored the application of Generative Adversarial Networks (GANs) in Image and Video Processing using TensorFlow. We covered data preprocessing, model building, training, and visualized the generated images. GANs are powerful tools for generating synthetic data that resembles the training data distribution, which can be used in various applications such as image synthesis, style transfer, and more.

## References

- [Generative Adversarial Networks (GANs): An Overview](https://arxiv.org/abs/1710.07035)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)